Version 11
Dated 23 Oct.

In this version we have changed all the steps into function. 
Also loop to go through result pages is proposed which can be cleaned furthur.
This is work in progress to show working code which will be cleaned as we increase the version count.



In [1]:
# import libraries
#import urllib2
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [2]:
def getsoupfromlink(quote_page):
    # specify the url
    quote_page = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'
    # query the website and return the html to the variable ‘page’
    page = urlopen(quote_page)
    # parse the html using beautiful soap and store in variable `soup`
    soup = BeautifulSoup(page, 'html.parser')
    return soup

#soup = getsoupfromlink(quote_page)

soup

In [3]:
#s = soup.text

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""


#print (find_between( s, "What is claimed is:", "* * * * *" ))
#print (find_between_r (s, '123', 'abc' ))

In [4]:
def extractlink (soup):
    alllinks = []
    count = 0 
    for link in soup.find_all('a', href=True):
        x =  link['href']
        if x[:19]=='/netacgi/nph-Parser' and x.find('Next') == -1:

            count = count+1
            alllinks.append ('http://patft.uspto.gov'+x)
    return alllinks

#alllinks = extractlink(soup)


In [5]:
def remove_duplicates(l):
    return list(set(l))

#alllinks = remove_duplicates(alllinks)

#print (len(alllinks))

In [22]:
def getdatafromlink(link, lstofp):
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    #quote_page2 = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=1&p=1&f=G&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&OS=ACLM/python&RS=ACLM/python/'
    quote_page2 = link
    
    page2 = urlopen(quote_page2)

    sleep(randint(8,15)/10)
    soupinfun = BeautifulSoup(page2, 'html.parser')
    
    try:
        heading= (soupinfun.find('font', {'size':'+1'}).text).replace("\n", "")
        heading = ' '.join(heading.split())
        abstract= (soupinfun.p.text).replace("\n", "")
        abstract = ' '.join(abstract.split())
        claims = (find_between (soupinfun.text, 'What is claimed is:', '* * * * *' )).replace("\n", "")
        claims = ' '.join(claims.split())
        #print soup
        lstofp.append([link,heading, abstract, claims])
        page2.close()
        #time.sleep(.1)

    except Exception:
        pass
    return lstofp;



In [7]:
def createdf(alllinks):
    listofp =[]
    for item in alllinks:
        listofp = getdatafromlink(item,listofp)

    cols = ['Link', 'Heading', 'Abstract', 'Claims']

    df1 = pd.DataFrame(listofp, columns=cols)

    df1.to_csv('output.txt', sep='|')
    return df1

#df1 = createdf(alllinks)

In [9]:
quote_page = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'

listofresultpages=[]

for num in range(1,9):
    leftlink= 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p='
    midnumber = num
    rightlink = '&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'
    quotepagejoined = leftlink+str(midnumber)+rightlink
    listofresultpages.append([quotepagejoined])

#print (quotepagejoined == quote_page)

#listofresultpages

In [10]:
quote_page = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'

for page in listofresultpages:
    soup = getsoupfromlink(page)
    alllinks = extractlink(soup)
    alllinks = remove_duplicates(alllinks)
    #print (alllinks)
    df1 = createdf(alllinks)
    print (df1)



                                                 Link  \
0   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
1   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
2   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
3   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
4   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
5   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
6   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
7   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
8   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
9   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
10  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
11  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
12  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
13  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
14  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
15  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
16  http://patft.uspto.gov/neta

In [ ]:
randint(8,15)/100

In [28]:
quote_page =['http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=1&f=S&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&Page=Prev&OS=ACLM/python&RS=ACLM/python', 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'
,'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=3&f=S&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&Page=Next&OS=ACLM/python&RS=ACLM/python']


for quote in quote_page:
    soup = getsoupfromlink(quote)
    alllinks = extractlink(soup)
    alllinks = remove_duplicates(alllinks)
    #print (alllinks)
    x = createdf(alllinks)
    #print (df1)
    dfmerged = dfmerged.add(x)

print (dfmerged)

                                                 Link  \
0   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
1   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
2   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
3   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
4   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
5   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
6   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
7   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
8   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
9   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
10  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
11  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
12  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
13  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
14  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
15  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
16  http://patft.uspto.gov/neta

In [17]:
for quote in quote_page:
    print(quote)

http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=1&f=S&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&Page=Prev&OS=ACLM/python&RS=ACLM/python
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT
http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=3&f=S&l=50&d=PTXT&S1=(python.CLTX.+or+python.DCTX.)&Page=Next&OS=ACLM/python&RS=ACLM/python


In [20]:
quote_page[1]

'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=0&p=2&f=S&l=50&Query=ACLM%2Fpython+&d=PTXT'

In [23]:

soup = getsoupfromlink(quote_page[1])
alllinks = extractlink(soup)
alllinks = remove_duplicates(alllinks)
#print (alllinks)
df1 = createdf(alllinks)
print (df1)


                                                 Link  \
0   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
1   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
2   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
3   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
4   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
5   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
6   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
7   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
8   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
9   http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
10  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
11  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
12  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
13  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
14  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
15  http://patft.uspto.gov/netacgi/nph-Parser?Sect...   
16  http://patft.uspto.gov/neta

In [27]:
dfmerged

,Link,Heading,Abstract,Claims
0,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Pattern modeling methods and systemsPattern mo...,A list of constructs is displayed on a user in...,1. A computer-implemented method for inserting...
1,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,System for translating diverse programming lan...,A computer language translator that translates...,1. A computer language translator for translat...
2,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Method and system for data centric heap profil...,A method and system for data centric heap prof...,"1. A method, implemented in a computing device..."
3,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Intelligent offline caching of non-navigated c...,A request for a navigated content associated w...,1. A system for usage-based offline caching co...
4,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Record based code structureRecord based code s...,The compiled code of a computer program is sto...,1. A system comprising: a computing device; op...
5,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Procedurally expressing graphic objects for we...,A graphics object can be expressed using proce...,
6,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Integrating map-reduce into a distributed rela...,A computer readable storage medium includes ex...,
7,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,Garbage collection implemented in hardwareGarb...,A computing device is provided and includes a ...,"1. A computing device, comprising: a memory mo..."
8,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,System and method for automatic hardware and s...,The present invention relates to a system and ...,1. A computer implemented method for defining ...
9,http://patft.uspto.gov/netacgi/nph-Parser?Sect...,"Program system, and method and system arrangem...",A method and system arrangement are disclosed ...,1. A method for generating a software applicat...


In [30]:
from collections import Counter

def splitplay (test):
    x = Counter(test.split()).most_common(1)
    return x[0][0]

dfmerged['freqofwords'] = dfmerged['Abstract'].apply(splitplay)

dfmerged['freqofwords'] 


0               the
1          computer
2                 a
3               can
4                of
5                 a
6        map-reduce
7                 a
8                 a
9           program
10              the
11              the
12                a
13               of
14                a
15                a
16                a
17    collaboration
18              and
19              the
20                a
21         function
22             node
23                a
24               to
25               to
26              the
27              the
28               in
29                a
30               to
31              the
32         document
33                a
34       subscriber
35         selected
36              the
37                a
38                a
39              the
40              and
41                a
42         computer
43              the
44              the
45                a
46         embedded
47                a
48                a
49               to
